# LangGraph Agent v1 — Basic Q&A

Single-node agent that answers directly with the LLM (no tools).

Quick guide:
- State: messages
- Node: analyze (answer_question) → END
- Observe prints: [ANSWER]
- Run the last cell to see a clean, single execution trace

In [ ]:
import os
from typing import List, Optional, Union, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field

# LangGraph
from langgraph.graph import StateGraph, END

# LangChain core
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_openai import ChatOpenAI
load_dotenv()

True

In [12]:
class AgentState(BaseModel):
    messages: List[Union[HumanMessage, AIMessage, SystemMessage]] = Field(description="The chat history")
    class Config:
        arbitrary_types_allowed = True

In [13]:
# Initialize LLM
#llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)


llm = ChatOpenAI(
  base_url="https://api.mistral.ai/v1",
  api_key=os.environ["MISTRAL_API_KEY"],
  model="mistral-large-2512",
)

In [14]:

def answer_question(state: AgentState):
    print("[ANSWER] Executing answer_question node…")

    user_query = next((m.content for m in reversed(state.messages) if isinstance(m, HumanMessage)), "")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a knowledgeable assistant. Your role is to answer the user question with the best of your knowledge."),
        ("human", "Question: {q}")
    ])
    ans = llm.invoke(prompt.format_messages(q=user_query)).content.strip()
    return {"messages": [AIMessage(content=ans)]}

In [15]:
# Build graph
workflow_v1 = StateGraph(AgentState)
workflow_v1.add_node("analyze", answer_question)


workflow_v1.set_entry_point("analyze")
workflow_v1.add_edge("analyze", END)


agent = workflow_v1.compile()

In [16]:
# Test v1
query = "What teams were in 2025 NBA playoffs?"
initial_state = AgentState(messages=[HumanMessage(content=query)])
final_state = agent.invoke(initial_state)


[ANSWER] Executing answer_question node…


In [17]:
for m in final_state['messages']:
    if isinstance(m, HumanMessage):
        print(f"Human: {m.content}")
    elif isinstance(m, AIMessage):
        print(f"AI: {m.content}")
    else:
        print(m)

AI: As of my last update in **October 2023**, I don’t have real-time or future-specific data, including the exact teams that will make the **2025 NBA playoffs**. However, I can provide some insights based on recent trends and projections:

### **Likely Contenders for the 2025 NBA Playoffs (Based on 2023-24 Season & Early Projections)**
The NBA playoffs typically feature the top **6 teams from each conference (East & West)** by regular-season record, with the **7th–10th seeds** competing in the **Play-In Tournament**.

#### **Eastern Conference (Potential 2025 Playoff Teams)**
1. **Milwaukee Bucks** (Giannis Antetokounmpo, Damian Lillard)
2. **Boston Celtics** (Jayson Tatum, Jaylen Brown)
3. **Philadelphia 76ers** (Joel Embiid, Tyrese Maxey)
4. **Cleveland Cavaliers** (Donovan Mitchell, Evan Mobley)
5. **New York Knicks** (Jalen Brunson, Julius Randle)
6. **Miami Heat** (Jimmy Butler, Bam Adebayo)
7. **Brooklyn Nets** (Mikal Bridges, potential young stars)
8. **Orlando Magic** (Paolo Ba

In [ ]:
final_state

{'messages': [AIMessage(content='As of my last update in **October 2023**, I don’t have real-time or future-specific data, including the exact teams that will make the **2025 NBA playoffs**. However, I can provide some insights based on recent trends and projections:\n\n### **Likely Contenders for the 2025 NBA Playoffs (Based on 2023-24 Season & Early Projections)**\nThe NBA playoffs typically feature the top **6 teams from each conference (East & West)** by regular-season record, with the **7th–10th seeds** competing in the **Play-In Tournament**.\n\n#### **Eastern Conference (Potential 2025 Playoff Teams)**\n1. **Milwaukee Bucks** (Giannis Antetokounmpo, Damian Lillard)\n2. **Boston Celtics** (Jayson Tatum, Jaylen Brown)\n3. **Philadelphia 76ers** (Joel Embiid, Tyrese Maxey)\n4. **Cleveland Cavaliers** (Donovan Mitchell, Evan Mobley)\n5. **New York Knicks** (Jalen Brunson, Julius Randle)\n6. **Miami Heat** (Jimmy Butler, Bam Adebayo)\n7. **Brooklyn Nets** (Mikal Bridges, potential yo

: 